In [1]:
import outliers_detect as od

In [2]:
dir(od)

['ABOD',
 'IsolationForest',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'iforest',
 'np',
 'pd',
 'plt',
 'sns']

In [3]:
PATH='C:/Users/98gud/Downloads/creditcard.csv/creditcard.csv'
df=od.pd.read_csv(PATH)


In [4]:
x=od.iforest()

In [5]:
y=x.iforest_index(df)

In [6]:
print(y)

[51, 82, 83, 132, 140, 146, 150, 164, 174, 210, 225, 290, 362, 370, 401, 455, 472, 529, 541, 601, 736, 841, 865, 952, 986, 1027, 1038, 1047, 1086, 1158, 1219, 1281, 1313, 1388, 1425, 1515, 1563, 1608, 1632, 1722, 1723, 1753, 1805, 1810, 1813, 1815, 1830, 1856, 1922, 2016, 2059, 2061, 2115, 2117, 2178, 2182, 2186, 2214, 2234, 2277, 2306, 2350, 2386, 2453, 2457, 2464, 2520, 2622, 2629, 2677, 2682, 2791, 2792, 2897, 2951, 2954, 2956, 2957, 2963, 3099, 3102, 3145, 3443, 3493, 3502, 3505, 3509, 3547, 3554, 3555, 3602, 3724, 3771, 3780, 4134, 4235, 4393, 4415, 4549, 4554, 4565, 4608, 4622, 4675, 4684, 4727, 4740, 4776, 4810, 4816, 4822, 4839, 4862, 4895, 4898, 4920, 4926, 4932, 4935, 4950, 5012, 5022, 5058, 5072, 5078, 5102, 5119, 5139, 5257, 5291, 5294, 5296, 5306, 5318, 5371, 5378, 5392, 5425, 5443, 5447, 5459, 5504, 5512, 5526, 5534, 5535, 5612, 5615, 5622, 5639, 5642, 5652, 5661, 5704, 5713, 5715, 5727, 5733, 5737, 5746, 5749, 5759, 5791, 5792, 5797, 5827, 5888, 5909, 5916, 5934, 5936, 5

In [7]:
z=x.iforest_score(df)

In [8]:
print(type(z))

<class 'numpy.ndarray'>


In [9]:
print(z)

[-0.36967038 -0.36323065 -0.46507245 ... -0.41270995 -0.44840691
 -0.39137623]


In [10]:
for i,value in enumerate(z):
    if i==1: break
    else: 
        print(value)
        print(type(value))

-0.3696703837763407
<class 'numpy.float64'>


In [11]:
z=od.pd.DataFrame(z,columns=['anomaly_score'])
z.describe()

,anomaly_score
count,284807.000000
mean,-0.397658
std,0.043834
min,-0.780910
25%,-0.412114
50%,-0.387074
75%,-0.369297
max,-0.340365


## iforest class 내의 매소드

1. init 을 통해서 파라미터 설정(contamination rate 변수로 받음, 입력 안하면 default 값)
2. init 설정으로 모델 적합시키고->
   데이터 프레임을 받아서 outlier index 값을 list 형으로 반환
3. 데이터 프레임을 받아서 outlier score값들을 ndarray형으로 반환
        -score의 range는 [-1,0] 
        -0에 가까울수록 거리가 짧고, anomaly로 감지될 가능성 大

### 추가되어야 할 것들:
- 그래프 그리기

### 의문인 것들:
- categorical 변수들은 어떡하나? 그냥 df에서 제외시켜야 하나?
